In [1]:
import pandas as pd
import json
filePath = r"C:\\Users\\Pat\\Desktop\\Projects\\LoLAramPredictor\\Game Info\\"

In [2]:
with open(filePath+'champion.json', encoding='utf8') as f:
    championData = json.load(f)

champions = pd.DataFrame(columns = ['championId', 'name', 'attack', 'defense', 'magic', 'difficulty', 'primaryClass', 'secondaryClass', 'partype', 'hp', 'hpperlevel', 'mp', 'mpperlevel', 'movespeed', 'armor', 'armorperlevel', 'spellblock', 'spellblockperlevel', 'attackrange', 'hpregen', 'hpregenperlevel', 'mpregen', 'mpregenperlevel', 'crit', 'critperlevel', 'attackdamage', 'attackdamageperlevel', 'attackspeedperlevel', 'attackspeed'])

for k, v in championData['data'].items():
    champions = champions.append({'championId': int(v['key']),
                        'name': v['name'], 
                        'attack': v['info']['attack'], 
                        'defense': v['info']['defense'], 
                        'magic': v['info']['magic'], 
                        'difficulty': v['info']['difficulty'], 
                        'primaryClass': v['tags'][0], 
                        'secondaryClass': v['tags'][1] if len(v['tags']) > 1 else 'None', 
                        'partype': v['partype'], 
                        'hp': v['stats']['hp'], 
                        'hpperlevel': v['stats']['hpperlevel'], 
                        'mp': v['stats']['mp'], 
                        'mpperlevel': v['stats']['mpperlevel'], 
                        'movespeed': v['stats']['movespeed'], 
                        'armor': v['stats']['armor'], 
                        'armorperlevel': v['stats']['armorperlevel'], 
                        'spellblock': v['stats']['spellblock'], 
                        'spellblockperlevel': v['stats']['spellblockperlevel'], 
                        'attackrange': v['stats']['attackrange'], 
                        'hpregen': v['stats']['hpregen'], 
                        'hpregenperlevel': v['stats']['hpregenperlevel'], 
                        'mpregen': v['stats']['mpregen'], 
                        'mpregenperlevel': v['stats']['mpregenperlevel'], 
                        'crit': v['stats']['crit'], 
                        'critperlevel': v['stats']['critperlevel'], 
                        'attackdamage': v['stats']['attackdamage'], 
                        'attackdamageperlevel': v['stats']['attackdamageperlevel'], 
                        'attackspeedperlevel': v['stats']['attackspeedperlevel'], 
                        'attackspeed': v['stats']['attackspeed']}, ignore_index=True)

champions

,championId,name,attack,defense,magic,difficulty,primaryClass,secondaryClass,partype,hp,...,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
0,266,Aatrox,8,4,3,4,Fighter,Tank,Blood Well,580,...,3,1,0,0,0,0,60,5,2.500,0.651
1,103,Ahri,3,4,8,5,Mage,Assassin,Mana,526,...,6.5,0.6,8,0.8,0,0,53.04,3,2.000,0.668
2,84,Akali,5,3,8,7,Assassin,None,Energy,575,...,8,0.5,50,0,0,0,62.4,3.3,3.200,0.625
3,12,Alistar,6,9,5,7,Tank,Support,Mana,600,...,8.5,0.85,8.5,0.8,0,0,62,3.75,2.125,0.625
4,32,Amumu,2,6,8,3,Tank,Mage,Mana,613.12,...,9,0.85,7.382,0.525,0,0,53.38,3.8,2.180,0.736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,238,Zed,9,2,1,7,Assassin,None,Energy,584,...,7,0.65,50,0,0,0,63,3.4,3.300,0.651
146,115,Ziggs,2,4,9,4,Mage,None,Mana,536,...,6.5,0.6,8,0.8,0,0,54.208,3.1,2.000,0.656
147,26,Zilean,2,5,8,6,Support,Mage,Mana,504,...,5.5,0.5,11.335,0.8,0,0,51.64,3,2.130,0.625
148,142,Zoe,1,7,8,5,Mage,Support,Mana,560,...,6.5,0.6,8,0.65,0,0,58,3.3,2.500,0.625


In [3]:
import asyncpg
import nest_asyncio
import asyncio
from config import dbConfig

nest_asyncio.apply()
loop = asyncio.get_event_loop()

In [4]:
supportNames = ['Alistar', 'Bard', 'Blitzcrank', 'Braum', 'Ivern', 'Janna', 'Karma', 'Leona', 'Lulu', 'Lux', 'Morgana', 'Nami', 'Nautilus', 'Pyke', 'Rakan', 'Senna', 'Sona', 'Soraka', 'Tahm Kench', 'Taric', 'Thresh', 'Yuumi', 'Zilean', 'Zyra']
supportChampIds = [champions.loc[champions['name'] == name, 'championId'].values[0] for name in supportNames]
# supportChampIds
print(f'supportChampIds({len(supportChampIds)}): {supportChampIds}')

supportChampIds(24): [12, 432, 53, 201, 427, 40, 43, 89, 117, 99, 25, 267, 111, 555, 497, 235, 37, 16, 223, 44, 412, 350, 26, 143]


In [5]:
import itertools
nonSupportNames = champions.loc[~(champions['name'].isin(supportNames)), 'name'].to_list()
nonSupportIds = champions.loc[~(champions['championId'].isin(supportChampIds)), 'championId'].to_list()
twoSupportNames = list(itertools.combinations(supportNames,2))
twoSupportIds = list(itertools.combinations(supportChampIds, 2))
threeSupportNames = list(itertools.combinations(supportNames,3))
threeSupportIds = list(itertools.combinations(supportChampIds, 3))
fourSupportNames = list(itertools.combinations(supportNames,4))
fourSupportIds = list(itertools.combinations(supportChampIds, 4))

print(nonSupportNames)
# print(nonSupportIds)
print(twoSupportNames)
# print(supportDuoIds)

['Aatrox', 'Ahri', 'Akali', 'Amumu', 'Anivia', 'Annie', 'Aphelios', 'Ashe', 'Aurelion Sol', 'Azir', 'Brand', 'Caitlyn', 'Camille', 'Cassiopeia', "Cho'Gath", 'Corki', 'Darius', 'Diana', 'Draven', 'Dr. Mundo', 'Ekko', 'Elise', 'Evelynn', 'Ezreal', 'Fiddlesticks', 'Fiora', 'Fizz', 'Galio', 'Gangplank', 'Garen', 'Gnar', 'Gragas', 'Graves', 'Hecarim', 'Heimerdinger', 'Illaoi', 'Irelia', 'Jarvan IV', 'Jax', 'Jayce', 'Jhin', 'Jinx', "Kai'Sa", 'Kalista', 'Karthus', 'Kassadin', 'Katarina', 'Kayle', 'Kayn', 'Kennen', "Kha'Zix", 'Kindred', 'Kled', "Kog'Maw", 'LeBlanc', 'Lee Sin', 'Lillia', 'Lissandra', 'Lucian', 'Malphite', 'Malzahar', 'Maokai', 'Master Yi', 'Miss Fortune', 'Wukong', 'Mordekaiser', 'Nasus', 'Neeko', 'Nidalee', 'Nocturne', 'Nunu & Willump', 'Olaf', 'Orianna', 'Ornn', 'Pantheon', 'Poppy', 'Qiyana', 'Quinn', 'Rammus', "Rek'Sai", 'Renekton', 'Rengar', 'Riven', 'Rumble', 'Ryze', 'Sejuani', 'Sett', 'Shaco', 'Shen', 'Shyvana', 'Singed', 'Sion', 'Sivir', 'Skarner', 'Swain', 'Sylas', 'Syn

In [6]:
async def fetch_as_dataframe(con: asyncpg.Connection, query: str, *args):
    stmt = await con.prepare(query)
    columns = [a.name for a in stmt.get_attributes()]
    data = await stmt.fetch()
    return pd.DataFrame(data, columns=columns)

async def run():
    connection = await asyncpg.connect(user=dbConfig['user'],
                                password=dbConfig['password'],
                                host=dbConfig['host'],
                                port=dbConfig['port'],
                                database=dbConfig['database'])
    async with connection.transaction():
        allGamesQuery = f'SELECT "gameId", "championId", "teamColor", "win" \
                    FROM "Participant_Stats" \
                    ORDER BY "gameId" ASC;'
        allGames_df = await fetch_as_dataframe(connection, allGamesQuery)

    return allGames_df

allGames_df = loop.run_until_complete(run())
allGames_df.head(10)

,gameId,championId,teamColor,win
0,2855142709,75,Blue,False
1,2855142709,22,Blue,False
2,2855142709,498,Red,True
3,2855142709,29,Red,True
4,2855142709,21,Blue,False
5,2855142709,122,Blue,False
6,2855142709,57,Red,True
7,2855142709,20,Blue,False
8,2855142709,117,Red,True
9,2855142709,267,Red,True


In [7]:
allGames_df['championId'] = pd.to_numeric(allGames_df['championId'], downcast='unsigned')
allGames_df['teamColor'] = allGames_df['teamColor'] == 'Red'
allGames_df.memory_usage(deep=True)

Index               128
gameId        129693160
championId     32423290
teamColor      16211645
win            16211645
dtype: int64

In [8]:
from multiprocessing.pool import ThreadPool as Pool
from statsmodels.stats.proportion import proportion_confint

In [9]:
twoSupports = [(nonSupportId, *supportId) for nonSupportId in nonSupportIds for supportId in twoSupportIds]
threeSupports = [(nonSupportId, *supportId) for nonSupportId in nonSupportIds for supportId in threeSupportIds]
fourSupports = [(nonSupportId, *supportId) for nonSupportId in nonSupportIds for supportId in fourSupportIds]
print(f'twoSupports({len(twoSupports)}): {twoSupports[:10]}')
print(f'threeSupports({len(threeSupports)}): {threeSupports[:10]}')
print(f'fourSupports({len(fourSupports)}):{fourSupports[:10]}')

twoSupports(34776): [(266, 12, 432), (266, 12, 53), (266, 12, 201), (266, 12, 427), (266, 12, 40), (266, 12, 43), (266, 12, 89), (266, 12, 117), (266, 12, 99), (266, 12, 25)]
threeSupports(255024): [(266, 12, 432, 53), (266, 12, 432, 201), (266, 12, 432, 427), (266, 12, 432, 40), (266, 12, 432, 43), (266, 12, 432, 89), (266, 12, 432, 117), (266, 12, 432, 99), (266, 12, 432, 25), (266, 12, 432, 267)]
fourSupports(1338876):[(266, 12, 432, 53, 201), (266, 12, 432, 53, 427), (266, 12, 432, 53, 40), (266, 12, 432, 53, 43), (266, 12, 432, 53, 89), (266, 12, 432, 53, 117), (266, 12, 432, 53, 99), (266, 12, 432, 53, 25), (266, 12, 432, 53, 267), (266, 12, 432, 53, 111)]


In [10]:
def calcWR(champs):
    allGames_df['isSynergy'] = allGames_df['championId'].isin(champs)
    synergyGames_df = allGames_df.loc[allGames_df['isSynergy'] == True, ['gameId', 'teamColor', 'win']]
    
    synergyGames_df = synergyGames_df.groupby(['gameId', 'teamColor']).agg(['sum', 'count'])
    synergyGames_df = synergyGames_df.loc[synergyGames_df['win']['count'] == 3, 'win']
    CI = proportion_confint(synergyGames_df['sum'].astype(bool).sum(axis=0), len(synergyGames_df.index), method='binom_test', alpha=0.05)
    # print(synergyGames_df)
    return {'nonSupportChamp': champions.loc[champions["championId"] == champs[0], "name"].values[0],
                                            'supportChamp1': champions.loc[champions["championId"] == champs[1], "name"].values[0],
                                            'supportChamp2': champions.loc[champions["championId"] == champs[2], "name"].values[0],  
                                            'winRate': synergyGames_df['sum'].astype(bool).sum(axis=0)/len(synergyGames_df.index),
                                            '95% CI': f"[{CI[0]:.6f}, {CI[1]:.6f}]",
                                            'numWins': synergyGames_df['sum'].astype(bool).sum(axis=0),
                                            'numGames': len(synergyGames_df.index)
                                            }

if __name__ == '__main__':
    pool = Pool(6)
    res = pool.map(calcWR, twoSupports)
    twoSynergy_df = pd.DataFrame.from_dict(res)

twoSynergy_df


,nonSupportChamp,supportChamp1,supportChamp2,winRate,95% CI,numWins,numGames
0,Aatrox,Alistar,Bard,0.285714,"[0.132448, 0.506052]",6,21
1,Aatrox,Alistar,Blitzcrank,0.372549,"[0.247436, 0.512552]",19,51
2,Aatrox,Alistar,Braum,0.470588,"[0.253004, 0.712627]",8,17
3,Aatrox,Alistar,Ivern,0.457143,"[0.297834, 0.630322]",16,35
4,Aatrox,Alistar,Janna,0.500000,"[0.357681, 0.642319]",23,46
...,...,...,...,...,...,...,...
34771,Zoe,Thresh,Zilean,0.341880,"[0.259994, 0.433106]",40,117
34772,Zoe,Thresh,Zyra,0.336538,"[0.249062, 0.432496]",35,104
34773,Zoe,Yuumi,Zilean,0.384615,"[0.240818, 0.551890]",15,39
34774,Zoe,Yuumi,Zyra,0.480769,"[0.344998, 0.616107]",25,52


In [11]:
twoSynergy_df.sort_values(axis=0, ascending=False, by='winRate', inplace=True)
twoSynergy_df.head(10)

,nonSupportChamp,supportChamp1,supportChamp2,winRate,95% CI,numWins,numGames
21822,Rek'Sai,Alistar,Taric,1.0,"[0.050000, 1.000000]",1,1
33933,Zac,Tahm Kench,Taric,1.0,"[0.050000, 1.000000]",1,1
32210,Xayah,Morgana,Thresh,1.0,"[0.223607, 1.000000]",2,2
20793,Poppy,Ivern,Nautilus,1.0,"[0.622858, 1.000000]",7,7
32803,Xin Zhao,Rakan,Taric,1.0,"[0.368403, 1.000000]",3,3
14418,Kled,Braum,Ivern,1.0,"[0.368403, 1.000000]",3,3
33494,Yorick,Ivern,Soraka,1.0,"[0.588641, 1.000000]",6,6
2443,Aurelion Sol,Rakan,Taric,1.0,"[0.050000, 1.000000]",1,1
8574,Gragas,Alistar,Taric,1.0,"[0.500000, 1.000000]",5,5
3546,Camille,Rakan,Tahm Kench,1.0,"[0.050000, 1.000000]",1,1


In [12]:
twoSynergy_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34776 entries, 21822 to 33496
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   nonSupportChamp  34776 non-null  object 
 1   supportChamp1    34776 non-null  object 
 2   supportChamp2    34776 non-null  object 
 3   winRate          34755 non-null  float64
 4   95% CI           34776 non-null  object 
 5   numWins          34776 non-null  int64  
 6   numGames         34776 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 2.1+ MB


In [13]:
twoSynergy_df.to_csv('twoSupports.csv', sep=',')

In [14]:
# anyTwoSupports = [(nonSupportId, *supportChampIds) for nonSupportId in nonSupportIds]
# anyTwoSupports[0]

In [15]:
def calcWR(champ):
    allGames_df['isTarget'] = allGames_df['championId'] == champ
    allGames_df['isSupport'] = allGames_df['championId'].isin(supportChampIds)
    synergyGames_df = allGames_df.loc[(allGames_df['isTarget'] == True) | (allGames_df['isSupport'] == True), ['gameId', 'teamColor', 'win']]
    synergyGames_df = synergyGames_df.groupby(['gameId', 'teamColor']).agg(['sum', 'count'])
    synergyGames_df = synergyGames_df.loc[synergyGames_df['win']['count'] == 3, 'win']
    targetGames_df = allGames_df.loc[allGames_df['isTarget'] == True, ['gameId', 'teamColor', 'win']]
    targetGames_df = targetGames_df.groupby(['gameId', 'teamColor']).agg(['sum'])['win']
    CI = proportion_confint(synergyGames_df['sum'].astype(bool).sum(axis=0), len(synergyGames_df.index), method='binom_test', alpha=0.05)
    baseCI = proportion_confint(targetGames_df['sum'].astype(bool).sum(axis=0), len(targetGames_df.index), method='binom_test', alpha=0.05)
    # targetGames_df = targetGames_df.loc[targetGames_df['win']['count'] == 1, 'win']
    return {'nonSupportChamp': champions.loc[champions["championId"] == champ, "name"].values[0],
                                            'winRate': synergyGames_df['sum'].astype(bool).sum(axis=0)/len(synergyGames_df.index),
                                            '95% CI': f"[{CI[0]:.6f}, {CI[1]:.6f}]",
                                            'numWins': synergyGames_df['sum'].astype(bool).sum(axis=0),
                                            'numGames': len(synergyGames_df.index),
                                            'baseWinRate': targetGames_df['sum'].astype(bool).sum(axis=0)/len(targetGames_df.index),
                                            'base 95% CI': f"[{baseCI[0]:.6f}, {baseCI[1]:.6f}]",
                                            'baseNumWins': targetGames_df['sum'].astype(bool).sum(axis=0),
                                            'baseNumGames': len(targetGames_df.index)
                                            }

if __name__ == '__main__':
    pool = Pool(6)
    res = pool.map(calcWR, nonSupportIds)
    anyTwoSupport_df = pd.DataFrame.from_dict(res)

anyTwoSupport_df
anyTwoSupport_df.sort_values(axis=0, ascending=False, by='winRate', inplace=True)
anyTwoSupport_df.head(10)

,nonSupportChamp,winRate,95% CI,numWins,numGames,baseWinRate,base 95% CI,baseNumWins,baseNumGames
47,Kayle,0.501831,"[0.498925, 0.504720]",57409,114399,0.538854,"[0.536351, 0.541352]",82374,152869
53,Kog'Maw,0.501831,"[0.498925, 0.504720]",57409,114399,0.538854,"[0.536351, 0.541352]",82374,152869
65,Mordekaiser,0.501831,"[0.498925, 0.504720]",57409,114399,0.538854,"[0.536351, 0.541352]",82374,152869
71,Olaf,0.501831,"[0.498925, 0.504720]",57409,114399,0.538854,"[0.536351, 0.541352]",82374,152869
41,Jinx,0.501831,"[0.498925, 0.504720]",57409,114399,0.538854,"[0.536351, 0.541352]",82374,152869
59,Malphite,0.501831,"[0.498925, 0.504720]",57409,114399,0.538854,"[0.536351, 0.541352]",82374,152869
32,Graves,0.500092,"[0.497190, 0.502994]",57121,114221,0.517175,"[0.514629, 0.519722]",76558,148031
20,Ekko,0.500092,"[0.497190, 0.502994]",57121,114221,0.517175,"[0.514629, 0.519722]",76558,148031
26,Fizz,0.500092,"[0.497190, 0.502994]",57121,114221,0.517175,"[0.514629, 0.519722]",76558,148031
110,Vel'Koz,0.498802,"[0.495924, 0.501681]",57878,116034,0.528160,"[0.525822, 0.530498]",92606,175337


In [16]:
anyTwoSupport_df.to_csv('anyTwoSupports.csv', sep=',')

In [17]:
def calcWR(champ):
    allGames_df['isTarget'] = allGames_df['championId'] == champ
    allGames_df['isSupport'] = allGames_df['championId'].isin(supportChampIds)
    synergyGames_df = allGames_df.loc[(allGames_df['isTarget'] == True) | (allGames_df['isSupport'] == True), ['gameId', 'teamColor', 'win']]
    synergyGames_df = synergyGames_df.groupby(['gameId', 'teamColor']).agg(['sum', 'count'])
    synergyGames_df = synergyGames_df.loc[synergyGames_df['win']['count'] >= 3, 'win']
    targetGames_df = allGames_df.loc[allGames_df['isTarget'] == True, ['gameId', 'teamColor', 'win']]
    targetGames_df = targetGames_df.groupby(['gameId', 'teamColor']).agg(['sum'])['win']
    # targetGames_df = targetGames_df.loc[targetGames_df['win']['count'] == 1, 'win']
    CI = proportion_confint(synergyGames_df['sum'].astype(bool).sum(axis=0), len(synergyGames_df.index), method='binom_test', alpha=0.05)
    baseCI = proportion_confint(targetGames_df['sum'].astype(bool).sum(axis=0), len(targetGames_df.index), method='binom_test', alpha=0.05)

    return {'nonSupportChamp': champions.loc[champions["championId"] == champ, "name"].values[0],
                                            'winRate': synergyGames_df['sum'].astype(bool).sum(axis=0)/len(synergyGames_df.index),
                                            '95% CI': f"[{CI[0]:.6f}, {CI[1]:.6f}]",
                                            'numWins': synergyGames_df['sum'].astype(bool).sum(axis=0),
                                            'numGames': len(synergyGames_df.index),
                                            'baseWinRate': targetGames_df['sum'].astype(bool).sum(axis=0)/len(targetGames_df.index),
                                            'base 95% CI': f"[{baseCI[0]:.6f}, {baseCI[1]:.6f}]",
                                            'baseNumWins': targetGames_df['sum'].astype(bool).sum(axis=0),
                                            'baseNumGames': len(targetGames_df.index)
                                            }

if __name__ == '__main__':
    pool = Pool(6)
    res = pool.map(calcWR, nonSupportIds)
    twoOrMoreSupports_df = pd.DataFrame.from_dict(res)

twoOrMoreSupports_df
twoOrMoreSupports_df.sort_values(axis=0, ascending=False, by='winRate', inplace=True)
twoOrMoreSupports_df.head(10)

,nonSupportChamp,winRate,95% CI,numWins,numGames,baseWinRate,base 95% CI,baseNumWins,baseNumGames
63,Miss Fortune,0.502042,"[0.499333, 0.504750]",65906,131276,0.532143,"[0.530000, 0.534276]",111511,209551
57,Lissandra,0.502042,"[0.499333, 0.504750]",65906,131276,0.532143,"[0.530000, 0.534276]",111511,209551
69,Nocturne,0.502042,"[0.499333, 0.504750]",65906,131276,0.532143,"[0.530000, 0.534276]",111511,209551
11,Caitlyn,0.498637,"[0.495946, 0.501329]",66233,132828,0.522673,"[0.520556, 0.524789]",111942,214172
23,Ezreal,0.498637,"[0.495946, 0.501329]",66233,132828,0.522673,"[0.520556, 0.524789]",111942,214172
17,Diana,0.498637,"[0.495946, 0.501329]",66233,132828,0.522673,"[0.520556, 0.524789]",111942,214172
5,Annie,0.498637,"[0.495946, 0.501329]",66233,132828,0.522673,"[0.520556, 0.524789]",111942,214172
35,Illaoi,0.498637,"[0.495946, 0.501329]",66233,132828,0.522673,"[0.520556, 0.524789]",111942,214172
29,Garen,0.498637,"[0.495946, 0.501329]",66233,132828,0.522673,"[0.520556, 0.524789]",111942,214172
26,Fizz,0.497243,"[0.494458, 0.500028]",61594,123871,0.517175,"[0.514629, 0.519722]",76558,148031


In [18]:
twoOrMoreSupports_df.to_csv('twoOrMoreSupports.csv', sep=',')

In [19]:
noSupGames_df = allGames_df[~(allGames_df['championId'].isin(supportChampIds))]
noSupGames = noSupGames_df[['gameId', 'teamColor', 'win']].groupby(['gameId', 'teamColor']).agg(['sum','count'])['win']
noSupGames

sum  count
gameId     teamColor            
2855142709 False        0      5
           True         3      3
2856401827 False        4      4
           True         0      5
2856437506 False        0      5
...                   ...    ...
3588313005 True         4      4
3588365616 False        0      5
           True         4      4
3588420238 False        0      4
           True         3      3

[3242145 rows x 2 columns]

In [20]:
supportGames_df = allGames_df[allGames_df['championId'].isin(supportChampIds)]
supportGames = supportGames_df[['gameId', 'teamColor', 'win']].groupby(['gameId', 'teamColor']).agg(['sum','count'])['win']
supportGames

sum  count
gameId     teamColor            
2855142709 True         2      2
2856401827 False        1      1
2856474922 False        1      1
           True         0      3
2856486557 False        0      1
...                   ...    ...
3588297793 True         0      2
3588313005 True         1      1
3588365616 True         1      1
3588420238 False        0      1
           True         2      2

[2074322 rows x 2 columns]

In [21]:
from statsmodels.stats.proportion import proportion_confint

In [22]:
noSup = noSupGames[noSupGames['count'] == 5]
oneSup = supportGames[supportGames['count'] == 1]
twoSup = supportGames[supportGames['count'] == 2]
threeSup = supportGames[supportGames['count'] == 3]
fourSup = supportGames[supportGames['count'] == 4]
fiveSup = supportGames[supportGames['count'] == 5]
atLeastOneSup = supportGames[supportGames['count'] >= 1]
twoOrMoreSup = supportGames[supportGames['count'] >= 2]
threeOrMoreSup = supportGames[supportGames['count'] >= 3]
fourOrMoreSup = supportGames[supportGames['count'] >= 4]

noSupWR = noSup['sum'].astype(bool).sum(axis=0)/noSup['count'].count()
oneSupWR = oneSup['sum'].astype(bool).sum(axis=0)/oneSup['count'].count()
twoSupWR = twoSup['sum'].astype(bool).sum(axis=0)/twoSup['count'].count()
threeSupWR = threeSup['sum'].astype(bool).sum(axis=0)/threeSup['count'].count()
fourSupWR = fourSup['sum'].astype(bool).sum(axis=0)/fourSup['count'].count()
fiveSupWR = fiveSup['sum'].astype(bool).sum(axis=0)/fiveSup['count'].count()
atLeastOneSupWR = atLeastOneSup['sum'].astype(bool).sum(axis=0)/atLeastOneSup['count'].count()
twoOrMoreSupWR = twoOrMoreSup['sum'].astype(bool).sum(axis=0)/twoOrMoreSup['count'].count()
threeOrMoreSupWR = threeOrMoreSup['sum'].astype(bool).sum(axis=0)/threeOrMoreSup['count'].count()
fourOrMoreSupWR = fourOrMoreSup['sum'].astype(bool).sum(axis=0)/fourOrMoreSup['count'].count()

noSupCI = proportion_confint(noSup['sum'].astype(bool).sum(axis=0), noSup['count'].count(), method='binom_test', alpha=0.05)
oneSupCI = proportion_confint(oneSup['sum'].astype(bool).sum(axis=0), oneSup['count'].count(), method='binom_test', alpha=0.05)
twoSupCI = proportion_confint(twoSup['sum'].astype(bool).sum(axis=0), twoSup['count'].count(), method='binom_test', alpha=0.05)
threeSupCI = proportion_confint(threeSup['sum'].astype(bool).sum(axis=0), threeSup['count'].count(), method='binom_test', alpha=0.05)
fourSupCI = proportion_confint(fourSup['sum'].astype(bool).sum(axis=0), fourSup['count'].count(), method='binom_test', alpha=0.05)
fiveSupCI = proportion_confint(fiveSup['sum'].astype(bool).sum(axis=0), fiveSup['count'].count(), method='binom_test', alpha=0.05)
atLeastOneSupCI = proportion_confint(atLeastOneSup['sum'].astype(bool).sum(axis=0), atLeastOneSup['count'].count(), method='binom_test', alpha=0.05)
twoOrMoreSupCI = proportion_confint(twoOrMoreSup['sum'].astype(bool).sum(axis=0), twoOrMoreSup['count'].count(), method='binom_test', alpha=0.05)
threeOrMoreSupCI = proportion_confint(threeOrMoreSup['sum'].astype(bool).sum(axis=0), threeOrMoreSup['count'].count(), method='binom_test', alpha=0.05)
fourOrMoreSupCI = proportion_confint(fourOrMoreSup['sum'].astype(bool).sum(axis=0), fourOrMoreSup['count'].count(), method='binom_test', alpha=0.05)

In [23]:
print(f"noSupWR: {noSupWR:.5f} [{noSup['sum'].astype(bool).sum(axis=0)} / {noSup['count'].count()}]")
print(f"oneSupWR: {oneSupWR:.5f} [{oneSup['sum'].astype(bool).sum(axis=0)} / {oneSup['count'].count()}]")
print(f"twoSupWR: {twoSupWR:.5f} [{twoSup['sum'].astype(bool).sum(axis=0)} / {twoSup['count'].count()}]")
print(f"threeSupWR: {threeSupWR:.5f} [{threeSup['sum'].astype(bool).sum(axis=0)} / {threeSup['count'].count()}]")
print(f"fourSupWR: {fourSupWR:.5f} [{fourSup['sum'].astype(bool).sum(axis=0)} / {fourSup['count'].count()}]")
print(f"fiveSupWR: {fourSupWR:.5f} [{fiveSup['sum'].astype(bool).sum(axis=0)} / {fiveSup['count'].count()}]")
print(f"atLeastOneSupWR: {atLeastOneSupWR:.5f} [{atLeastOneSup['sum'].astype(bool).sum(axis=0)} / {atLeastOneSup['count'].count()}]")
print(f"twoOrMoreSupWR: {twoOrMoreSupWR:.5f} [{twoOrMoreSup['sum'].astype(bool).sum(axis=0)} / {twoOrMoreSup['count'].count()}]")
print(f"threeOrMoreSupWR: {threeOrMoreSupWR:.5f} [{threeOrMoreSup['sum'].astype(bool).sum(axis=0)} / {threeOrMoreSup['count'].count()}]")
print(f"fourOrMoreSupWR: {fourOrMoreSupWR:.5f} [{fourOrMoreSup['sum'].astype(bool).sum(axis=0)} / {fourOrMoreSup['count'].count()}]")

print(f"95% CI: [{noSupCI[0]:.6f}, {noSupCI[1]:.6f}]")
print(f"95% CI: [{oneSupCI[0]:.6f}, {oneSupCI[1]:.6f}]")
print(f"95% CI: [{twoSupCI[0]:.6f}, {twoSupCI[1]:.6f}]")
print(f"95% CI: [{threeSupCI[0]:.6f}, {threeSupCI[1]:.6f}]")
print(f"95% CI: [{fourSupCI[0]:.6f}, {fourSupCI[1]:.6f}]")
print(f"95% CI: [{fiveSupCI[0]:.6f}, {fiveSupCI[1]:.6f}]")
print(f"95% CI: [{atLeastOneSupCI[0]:.6f}, {atLeastOneSupCI[1]:.6f}]")
print(f"95% CI: [{twoOrMoreSupCI[0]:.6f}, {twoOrMoreSupCI[1]:.6f}]")
print(f"95% CI: [{threeOrMoreSupCI[0]:.6f}, {threeOrMoreSupCI[1]:.6f}]")
print(f"95% CI: [{fourOrMoreSupCI[0]:.6f}, {fourOrMoreSupCI[1]:.6f}]")

noSupWR: 0.48858 [570660 / 1168007]
oneSupWR: 0.50705 [710253 / 1400748]
twoSupWR: 0.50826 [289621 / 569828]
threeSupWR: 0.49196 [47514 / 96581]
fourSupWR: 0.43973 [3068 / 6977]
fiveSupWR: 0.43973 [48 / 188]
atLeastOneSupWR: 0.50643 [1050504 / 2074322]
twoOrMoreSupWR: 0.50514 [340251 / 673574]
threeOrMoreSupWR: 0.48802 [50630 / 103746]
fourOrMoreSupWR: 0.43489 [3116 / 7165]
95% CI: [0.487669, 0.489483]
95% CI: [0.506225, 0.507881]
95% CI: [0.506962, 0.509559]
95% CI: [0.488807, 0.495118]
95% CI: [0.428046, 0.451410]
95% CI: [0.190846, 0.317268]
95% CI: [0.505752, 0.507113]
95% CI: [0.503948, 0.506337]
95% CI: [0.484973, 0.491065]
95% CI: [0.423444, 0.446404]
